# Constrained optimization : convex methods and other approaches

## Resources

Machine learning and optimization:
https://towardsdatascience.com/a-quick-overview-of-optimization-models-for-machine-learning-and-statistics-38e3a7d13138

Convex functions : https://www.youtube.com/watch?v=7QmGj1_i3MU

*Hands on Machine Learning ...*, Chapter 4 : Gradient Descent : https://drive.google.com/file/d/1t0rc3x5YQBgLXVLET6BzR4jn5vzMI_m0/view?usp=sharing

CVXPY documentation:
https://www.cvxpy.org/

The knapsack problem : 
https://en.wikipedia.org/wiki/Knapsack_problem

NP-Hardness :
https://en.wikipedia.org/wiki/NP-hardness

CVXPY advanced functionalities for integer problems :
https://www.cvxpy.org/tutorial/advanced/index.html


Penalty method : https://en.wikipedia.org/wiki/Penalty_method

Keras regularizers documentation:
https://keras.io/api/layers/regularizers/

Keras constraints documentation:
https://keras.io/api/layers/constraints/


------------------------------------------------
------------------------------------------------

**More about Convex Optimization:**

Stephen Boyd's MOOC:
https://www.edx.org/course/convex-optimization

Stephen Boyd's book:
https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf




In [ ]:
import numpy as np
import pandas as pd
import cvxpy as cp

# Part 1 : Convex constrained optimization

Sometimes when adressing a practical data science problem, you might be asked to produce a model that respect some constraints. For example produce a linear regression, but only with positive coefficients because it doesn't make sense to have negative coefficients for the problem you are adressing. 

As you know by now, most of machine learning consists in minimizing an error function. Minimizing a function is the definition of an applied mathematics field : *optimization*. Thus most of machine learning actually is optimization, you can read more about this here:

https://towardsdatascience.com/a-quick-overview-of-optimization-models-for-machine-learning-and-statistics-38e3a7d13138


The general form of an optimization problem (or program) also allows to impose constraints on the function parameters. It writes:

<div style="font-size: 150%;" align= "center"> 
\begin{eqnarray*}
    \mathrm{minimize} &\;\;&f_0(x)&\;&\\
    \mathrm{subject\ to}&\;\;&f_i(x) \leq 0 &\;& i = 1, \ldots, m\\
    &&h_j(x) = 0 &\;& j = 1, \ldots, p
\end{eqnarray*}
</div>

- $x \in \mathbb{R}^n$ is the optimization variable, or the parameters
- $f_0 : \mathbb{R}^n \to \mathbb{R}$ is the objective or error function
- $f_i : \mathbb{R}^n \to \mathbb{R}$ are the inequality constraint functions
- $h_i : \mathbb{R}^n \to \mathbb{R}$ are the equality constraint functions

The optimal value $p^\star$ of $f_0$ is reached for the parameters $x$ that satisfies all the constraints and minimize $f_0$ : $p^\star = \mathrm{inf}\{ f_0(x) | f_i(x) \leq 0, i = 1, \ldots, m, h_j(x) = 0, j = 1, \ldots, p$ \}. Such optimal value of the parameters $x$ is written $x^\star$.

Sometimes when the constraints are incompatible, there are no solutions and the problem is *infeasible*. Conversely sometimes $p^\star = - \infty $ and the problem is called *unbounded below*.

Not all optimization problems are easy to solve, but a subset of them, called *convex optimization problems*, are. A convex optimization problem follows this form :

<div style="font-size: 150%;" align= "center"> 
\begin{eqnarray*}
    \mathrm{minimize} &\;\;&f_0(x)&\;&\\
    \mathrm{subject\ to}&\;\;&f_i(x) \leq 0 &\;& i = 1, \ldots, m\\
    &&a_j^T x = b_j &\;& j = 1, \ldots, p
\end{eqnarray*}
</div>

where $f_0, \cdots, f_m$ are convex functions, and equality constraints are affine.

A function is *convex* if the line segment between any two points on the graph of the function lies above the graph between the two points. Seeing it drawn makes it much clearer : https://www.youtube.com/watch?v=7QmGj1_i3MU

Convex optimization problems have the nice property that any locally optimal point is also a globally optimal point $x^\star$, which makes them relatively easy to solve and guarantees that the solution $x^\star$ is among the best ones.

If you don't remember what are local and global optima, re-read the *Gradient descent* part in chapter 4 of *Hands on Machine Learning... * : https://drive.google.com/file/d/1t0rc3x5YQBgLXVLET6BzR4jn5vzMI_m0/view?usp=sharing

Using the CVXPY library, you will learn how to solve such constrained convex problems.

## Linear problems : The diet problem

Let's start with a classic problem called the diet problem. We have:
- $n$ different types of food
- $m$ different types of nutrients
- The food costs $c \in \mathbb{R}^n$
- The matrix of nutrient content for each type of food $A \in \mathbb{R}^{m \times n}$
- The minimum amount of nutrients needed for a human being $b \in \mathbb{R}^m$

We want to minimize the cost in food spendings such that the corresponding diet satisfies all the nutrient needs. In other words, we want to :

<div style="font-size: 150%;" align= "center"> 
\begin{eqnarray*}
    \mathrm{minimize} &\;\;& c^T x\\
    \mathrm{subject\ to}&\;\;&Ax \geq b\\
    &&x \geq 0
\end{eqnarray*}
</div>

The solution $x^\star \in \mathbb{R}^n$ gives us the optimal quantity of each food to buy (and thus cannot be negative).

Let's start easy and forget about the nutrients, skim through CVXPY documentation : https://www.cvxpy.org/ ,
and solve the following problem:

<div style="font-size: 150%;" align= "center"> 
\begin{eqnarray*}
    \mathrm{minimize} &\;\;& c^T x\\
    &&x \geq 0
\end{eqnarray*}
</div>

In [ ]:
foods = ["Roasted Chicken", "Spaghetti", "Tomato,Red,Ripe,Raw", "Apple,Raw,W/Skin", 
         "Grapes", "Chocolate Chip Cookies", "Lowfat Milk", "Hotdog"]

food_costs = pd.Series([0.84, 0.78, 0.27, 0.24, 0.32, 0.03, 0.23, 0.31], index = foods)


In [ ]:
#TOFILL
n = len(foods)

c = food_costs.to_numpy()

# Define and solve the CVXPY problem.
x = cp.Variable(n)

objective = cp.Minimize(c.T@x)
constraints = [ x >= 0 ]

prob = cp.Problem(objective, constraints)
prob.solve()

# Print result.
print("\nThe optimal value is :", prob.value)
print("A solution x is :")
print( pd.Series(x.value, index = foods))

No surprises, without nutrient constraints, the optimal solution to minimize the cost is to stop eating ! Now add the $Ax \geq b$ constraint with the following data :

In [ ]:
nutrients = ["Calories", "Calcium", "Iron", "Vit_A", "Dietary_Fiber", "Carbohydrates", "Protein"]

food_nutrients = pd.DataFrame([
    (277.4, 21.9, 1.8, 77.4, 0, 0, 42.2),
    (358.2, 80.2, 2.3, 1355.2, 11.6, 58.3, 8.2),
    (25.8, 6.2, 0.6, 766.3, 1.4, 5.7, 1),
    (81.4, 9.7, 0.2, 73.1, 3.7, 21, 0.3),
    (15.1, 3.4, 0.1, 24, 0.2, 4.1, 0.2),
    (78.1, 6.2, 0.4, 0, 0, 24, 0.9),
    (121.2, 296.7, 0.1, 500.2, 0, 11.7, 8.1),
    (242.1, 23.5, 2.3, 0, 0, 18, 10.4) ],
    columns = nutrients,
    index = foods).T

nutrients_min = pd.Series([2000, 800, 10, 10000, 25, 0, 50], index = nutrients)

In [ ]:
#TOFILL
n = len(foods)
m = len(nutrients)

c = food_costs.to_numpy()
A = food_nutrients.to_numpy()
b = nutrients_min.to_numpy()

# Define and solve the CVXPY problem.
x = cp.Variable(n)

objective = cp.Minimize(c.T@x)
constraints = [ x >= 0 , A@x >= b]

prob = cp.Problem(objective, constraints)
prob.solve()

# Print result and corresponding number of each nutrient
print("\nThe optimal value is :", prob.value)
print("A solution x is :")
print( pd.Series(x.value, index = foods))

print("The corresponding nutrients quantities are :")
print( pd.Series(A.dot(x.value), index = nutrients))

This doesn't look like a very equilibrated diet, especially regarding carbohydrates, let's add some maximum constraints on nutrients : $Ax \leq d$, where  $d \in \mathbb{R}^m$ is the maximum amount of nutrients :

In [ ]:
nutrients_max = pd.Series([3000, 1600, 30, 50000, 100, 250, 100], index = nutrients)

In [ ]:
#TOFILL
n = len(foods)
m = len(nutrients)

c = food_costs.to_numpy()
A = food_nutrients.to_numpy()
b = nutrients_min.to_numpy()
d = nutrients_max.to_numpy()

# Define and solve the CVXPY problem.
x = cp.Variable(n)

objective = cp.Minimize(c.T@x)
constraints = [ x >= 0 , A@x >= b, A@x <= d]

prob = cp.Problem(objective, constraints)
prob.solve()

# Print result and corresponding number of each nutrient
print("\nThe optimal value is :", prob.value)
print("A solution x is :")
print( pd.Series(x.value, index = foods))

print("The corresponding nutrients quantities are :")
print( pd.Series(A.dot(x.value), index = nutrients))

So one can basically live on spaghetti, tomatoes, cookies, milk and hotdogs. Good to know.

## Quadratic problems : Constrained linear regression

Now let's switch to a classic machine learning model : linear regression, but with constraints on the regression coefficients.

In the following example, we have $n$ chemical elements, $m$ minerals, a matrix $A \in \mathbb{R}^{n \times m}$ that gives the chemical composition of each mineral, and the chemical composition of a geological sample $b \in \mathbb{R}^m$. 

From this we want to approximate the mineral composition of the geological sample $x \in \mathbb{R}^m$ with a linear regression. Thus $x$ must contain only non-negative values, and must sum to 1.

Write the corresponding convex problem on paper, and then with CVXPY using the following data:

In [ ]:
# 10 Chemical element x 13 mineral type matrix, containing the proportion of each element in each mineral type

mineral_chemical_compositions = np.array([[63.1545, 64.3049, 100., 37.1417, 32.4026, 30.0382, 30.7033, 0., 36.5444, 0., 0., 0., 0.0034],
   [0.0016, 0.01, 0., 0.6641, 2.35946, 26.91, 0., 0., 0.297125, 0., 0., 0., 51.3026], 
   [17.8683, 21.096, 0., 11.2387, 14.3019, 6.46115, 0.34805, 0., 14.368, 0., 0., 0., 0.], 
   [0.0223, 0.1191, 0., 30.6859, 32.0779, 1.5471, 1.5014, 0., 11.5299, 0., 0., 0., 43.3621], 
   [0., 0., 0., 0.94408, 0.6131, 0.0986, 0., 0., 0., 0., 0., 0., 2.1039], 
   [0., 0., 0., 1.14598, 1.82486, 0.03695, 0.00935, 0., 0.2312, 0., 0., 0., 0.0166], 
   [0.625667, 9.77073, 0., 1.35584, 0.0558, 0.05635, 0., 0., 0.079275, 0., 0., 0., 0.018], 
   [0., 2.80603, 0., 10.7388, 0.0245429, 27.7454, 2.19443, 55.08, 10.5478, 56.03, 78.13, 10.44, 0.0262], 
   [15.7365, 0.0874667, 0., 1.89652, 8.54851, 0.00475, 0., 0., 0.09665, 0., 0., 0., 0.0115], 
   [0., 0., 0., 0.16636, 0., 0., 0.0293, 42.4, 0.0114, 0., 0., 0., 0.0617]])

# Proportion of each of the 10 elements analyzed in the geological sample

sample_chemical_composition = np.array([ 65.67, 0.52, 14.77, 5.418, 0.13, 0.3, 3.22, 2.05, 6.07, 0.13 ])

In [ ]:
#TOFILL
A = mineral_chemical_compositions
b = sample_chemical_composition

n = A.shape[1]
m = A.shape[0]

x = cp.Variable(n)
objective = cp.Minimize(cp.sum_squares(A@x - b))
constraints = [0 <= x, cp.sum(x) == 1]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()

# The optimal value for x is stored in `x.value`.

print("\nThe optimal value is :", prob.value)
print("A solution x is :")
print(x.value)
cvx_cstr_reg_x_opt = x.value.copy()

## Integer problems : the knapsack problem

The knapsack problem is a classic combinatorial optimization problem, where one must maximize the total value of the objects put in the knapsack, without exceeding the knapsack capacity.

Read more about the knapsack problem : https://en.wikipedia.org/wiki/Knapsack_problem

The knapsack problem is called an integer problem as some variable values are constrained to integer values. Here we are going to address the 0-1 knapsack, where we can put only once each object in the knapsack. Problems where variable values are constrained to be 0 or 1 are a subtype of integer problems called boolean problems.

All integer problems are non-convex problems, and we do not know any algorithms to solve them in polynomial time (as they are NP-Hard problems https://en.wikipedia.org/wiki/NP-hardness ).

However CVXPY integrates Mixed-Integer solvers that yields good approximations of such problems in a reasonable time.

Use these functionalities of cvxpy to solve the following 0-1 knapsack problem :
https://www.cvxpy.org/tutorial/advanced/index.html

In [ ]:
object_values = np.array([10, 8, 1, 5.5, 3, 12, 2])
object_weights = np.array([5, 3, 0.1, 5, 4, 18, 5])

knapsack_max_weight = 15

In [ ]:
#TOFILL
n = len(object_values)

w = object_weights
v = object_values

# Define and solve the CVXPY problem.
x = cp.Variable(n, boolean =  True)

objective = cp.Maximize(v.T@x)
constraints = [ w.T@x <= knapsack_max_weight]

prob = cp.Problem(objective, constraints)
prob.solve()

# Print result and corresponding number of each nutrient
print("\nThe optimal value is :", prob.value)
print("A solution x is :")
print(x.value)


However, even these solvers become too slow when the problem becomes too big. When a non-convex problem is untractable, a classical technique is to relax its constraints in order to build an approximation of the problem that is convex. This is called a *convex relaxation*.

Propose a convex relaxation of the knapsack problem for the following data, and solve it with cvxpy:

In [ ]:
#Creating n random object values and weights between 0 and 10
n = 1000000
scale = 10

object_values = np.array([np.random.random() * scale for i in range(n) ])
object_weights = np.array([np.random.random() * scale for i in range(n) ])

knapsack_max_weight = (n * scale) / 20

In [ ]:
#TOFILL
w = object_weights
v = object_values

# Define and solve the CVXPY problem.
x = cp.Variable(n)

objective = cp.Maximize(v.T@x)
constraints = [ w.T@x <= knapsack_max_weight, x >= 0, x <= 1]

prob = cp.Problem(objective, constraints)
prob.solve()

# Print result and corresponding number of each nutrient
print("\nThe optimal value is :", prob.value)
print("A solution x is :")
print(x.value[:50])


As you can see, this yields a useful result, even if the original constraints are not exactly satisfied.

However depending on the problem constraint values, sometimes such approximations doesn't give a very useful result.

# [Optional] Re-assigning electricity production units to each production plant

In this problem you have $k$ electricity production plants, in which there are a total of $n_o$ production units, and for each of them you have its production history as a time series of length *m*. Unfortunately, you don't know which unit belongs to which plant, and some units production history are missing (you only have $n$ units history available), but you know the total production history of each plant. Given these informations, re-assign each unit to its production plant. When you're done try to change the number of plants and the missing ratio to see how far your solution correctly works.

In [ ]:
#Problem generation :

# Number of electricity production units
n_o = 100 
# Number of time steps
m = 5000
# Number of electricity production plants
k = 2
# Ratio of missing production units data
missing_ratio = 0.1

#Production history for each production unit 
unit_productions = np.random.random((n_o,m)) * 10

#Ground truth affectation of each production unit to a production plant
affectations = np.random.choice(k, n_o)

# Number of available production units data
n = int(n_o*(1-missing_ratio)) 

#Available production units 
available_units_indexes = np.random.choice(n_o, n, replace=False)
available_units_affectation = affectations[available_units_indexes]

In [ ]:
#Available data :

#Total production of each plant
total_productions = np.vstack( 
    [ unit_productions[affectations == i].sum(axis=0) for i in range(k) ])

#Available production units data
unit_productions_available = unit_productions[available_units_indexes]

#Goal try to find back in which plant is each production unit only from available data,
#i.e. try to find back the `available_units_affectation` array 

In [ ]:
#TOFILL
x = cp.Variable((k,n))

w = unit_productions_available
c = total_productions

objective = cp.Minimize(cp.sum(( x@w - c)))
constraints = [ x@w <= c, x >= 0, x <= 1, cp.sum(x, axis=0) == 1]

prob = cp.Problem(objective, constraints)
prob.solve()

# Print result
print("\nThe optimal value is :", prob.value)
print("A solution x is :")
print(x.value[:50])


In [ ]:
import matplotlib.pyplot as plt
plt.imshow(x.value)

In [ ]:
plt.imshow(available_units_affectation.reshape((1,available_units_affectation.size)))


What is the link between this problem and the knapsack problem ?

# Part 2 : Other constrained optimization methods

Sometimes when a constrained problem is not convex, or when it is to big to be solved with a convex solver, it can be useful to turn to other techniques to approximately enforce these constraints, such as using penalty functions, or projected gradient descent.

## Constrained linear regression (again)

In the following and to keep a continuity in the course, we will continue with our constrained linear regression problem to approximate the mineral composition of a geological sample, even if in this case, using a convex solver should be preferred as it is a convex problem !

### Replacing constraints with penalties

The first method consists in adding penalties to the error function $f_0$ that correspond to the constraints.

Read https://en.wikipedia.org/wiki/Penalty_method
 
In our case we will use linear penalties to replace :
- inequality constraints $f_i(x) \leq 0 $ by adding in the objective function : $ + \lambda\;\max(0,f_i(x))$
- equality constraints $h_j(x) = 0 $ by adding in the objective function : $ + \lambda |h_j(x)| $

where $\lambda \in \mathbb{R}$ is a new hyperparameter of the model, and controls the importance of the constraints relatively to the original objective function $f_0$.

Hence when replacing constraints with linear penalties the problem becomes :


<div style="font-size: 150%;" align= "center"> 
$\mathrm{minimize} \;\; f_0(x) + \lambda(\max(0,f_i(x)) + |h_j(x)|)$
</div>


First write on paper the corresponding optimization problem when replacing constraints with such penalties for the constrained linear regression problem of the geological sample composition seen before.

To implement this penalized version of the constrained linear regression problem, we will use Keras. Implement first a classic linear regression with Keras without any penalty (and don't forget to not fit the intercept, as we don't want it in our geological example !) :

In [ ]:
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *

In [ ]:
A = mineral_chemical_compositions
b = sample_chemical_composition

In [ ]:
def linear_regression_model(input_dim):
    """
    Input : 
    input_dim : int : Number of features of the linear regression
    
    Output : 
    model : tf.keras.Model : the keras Model object
    regression_layer : tf.keras.layers.Layer : the keras Layer object 
                        that contains the linear regression coefficients
                        
    """
    #TOFILL
    model = Sequential()
    regression_layer = Dense(1, activation = 'linear', input_dim = input_dim, use_bias = False)
    model.add(regression_layer)
    model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error', metrics = ['mse'])
    
    #TOKEEP
    return model, regression_layer

In [ ]:
max_epochs = 2000
batch_size = 10

model, regression_layer = linear_regression_model(A.shape[1])
model.fit(A,b, epochs=max_epochs, batch_size=batch_size)

In [ ]:
regression_layer.get_weights()

As we can see, our desired constraints (the coefficients $x \in \mathbb{R}^n$ must be only non-negative values, and must sum to 1) are not naturally respected.

Let's first add the value constraints : $ x \geq 0$ and $x \leq 1$, by adding the corresponding penalties : $\lambda\;\sum_i\max(0, -x_i + 0)$ and $\lambda\;\sum_i\max(0,x_i - 1 )$.

To do so we will use a custom layer weight regularizer.
Learn about layer regularizers : https://keras.io/api/layers/regularizers/


Let's code a regularizer that add penalties for any minimum and maximum values and not just 0 and 1.
Complete the following code to add the corresponding penalties to the layer weights :

In [ ]:
from tensorflow.keras import regularizers
import tensorflow as tf


class LinearBoxPenalties(regularizers.Regularizer):

    def __init__(self, lambda_, min_value, max_value):
        self.lambda_ = lambda_
        self.min_value = min_value
        self.max_value = max_value
        
    def get_config(self):
        return {'lambda': self.lambda_, 'min_value' : self.min_value, 'max_value' : self.max_value}

    def __call__(self, x):
        #TOFILL:
        return self.lambda_ * (tf.reduce_sum(tf.nn.relu(x - self.max_value)) 
                               + tf.reduce_sum(tf.nn.relu(-x + self.min_value))) 

    
    


And rewrite your linear regression so that it takes into account your penalty :

In [ ]:
def linear_regression_model_with_penalty_cstrs(input_dim, regularizer):
    """
    Input : 
    input_dim : int : Number of features of the linear regression
    regularizer : tf.keras.regularizers.Regularizer : The regularizer object to add to linear regression coefficients
    
    Output : 
    model : tf.keras.Model : the keras Model object
    regression_layer : tf.keras.layers.Layer : the keras Layer object 
                        that contains the linear regression coefficients
                        
    """
    #TOFILL
    model = Sequential()
    regression_layer = Dense(1, activation = 'linear', input_dim = input_dim,
                            use_bias = False, kernel_regularizer = regularizer)
    model.add(regression_layer)
    model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error', metrics = ['mse'])
    
    #TOKEEP
    return model, regression_layer

Now retrain your linear regression with your penalty and compare the obtained coefficients with ones you obtained with cvxpy :

In [ ]:
penalty_regularizer = LinearBoxPenalties(lambda_ = 10, min_value = 0, max_value = 1)

model, regression_layer = linear_regression_model_with_penalty_cstrs(A.shape[1], penalty_regularizer)

model.fit(A,b, epochs = max_epochs, batch_size = batch_size)

In [ ]:
#TOFILL
regression_layer.get_weights()[0].flatten()

In [ ]:
regression_layer.get_weights()[0].sum()

In [ ]:
cvx_cstr_reg_x_opt

We can see that the min and max constraints are well satisfied, however the coefficient still doesn't sum to one. 

Complete the following code to add both a min and max values penalties, as well as the sum penalty $ \lambda |\sum_i(x_i) - 1|$ :

In [ ]:
from tensorflow.keras import regularizers
import tensorflow as tf


class LinearBoxAndSumPenalties(regularizers.Regularizer):

    def __init__(self, lambda_, min_value, max_value, sum_value):
        self.lambda_ = lambda_
        self.min_value = min_value
        self.max_value = max_value
        self.sum_value = sum_value
        
    def get_config(self):
        return {'lambda': self.lambda_, 'min_value' : self.min_value, 
                'max_value' : self.max_value, 'sum_value' : self.sum_value }
    

    def __call__(self, x):
        #TOFILL
        minmax_penalty = tf.reduce_sum(tf.nn.relu(x - self.max_value)) + tf.reduce_sum(tf.nn.relu(-x + self.min_value))
        sum_penalty = tf.abs(tf.reduce_sum(x) - self.sum_value)
        
        return self.lambda_ * (minmax_penalty + sum_penalty)
        

    


In [ ]:
penalty_regularizer = LinearBoxAndSumPenalties(lambda_ = 10, min_value = 0, max_value = 1, sum_value = 1)

model, regression_layer = linear_regression_model_with_penalty_cstrs(A.shape[1], penalty_regularizer)

model.fit(A,b, epochs=max_epochs, batch_size=batch_size)

In [ ]:
#TOFILL
regression_layer.get_weights()[0].flatten()

In [ ]:
regression_layer.get_weights()[0].sum()

In [ ]:
cvx_cstr_reg_x_opt

Play with lambda and see how it changes the obtained $x$ values compared to the result of the convex approach.

### Enforcing constraints with projected gradient descent

The other main method to use constraints with problems that cannot be solved by convex solvers is called *projected gradient descent*. The idea is pretty simple, we only $\mathrm{minimize}\;f_0(x)$ with gradient descent, and after each gradient update, we have a look at the parameters, if they do not respect the constraints, then we modify them so that they respect the constraints. In other words we project the parameters values on the surface of the parameter-values set that respect the constraints.

In our case that means that if any coefficient in $x$ is below 0, we replace it by zero ; if any coefficient in $x$ is above 1, we replace it by one ; and if $x$ doesn't sum to 1, we rescale all the coefficients in $x$ so that they do.

Again we can do this in Keras, with the layer weights constraints : https://keras.io/api/layers/constraints/

Let's code a constraint that replaces the layer wieghts below a minimum value by this value, and above a maximum value by this other value.

Complete the following code to add these projection steps to your layer weights :

In [ ]:
from keras.constraints import Constraint
import keras.backend as K

class ProjectedBoxConstraint(Constraint):
    def __init__(self, min_value, max_value):
        self.min_value = min_value
        self.max_value = max_value
        
    def get_config(self):
        return {'min_value': self.min_value,
                'max_value': self.max_value}
    
    def __call__(self, w):
        #TOFILL
        clipped_w = K.clip(w, self.min_value, self.max_value) 
        return clipped_w 
    

In [ ]:
def linear_regression_model_with_projected_cstrs(input_dim, cstr):
    """
    Input : 
    input_dim : int : Number of features of the linear regression
    cstr : tf.keras.constraints.Constraint : the keras Constraint object 
                        that contains the projection operations to satisfy the constraints
    
    Output : 
    model : tf.keras.Model : the keras Model object
    regression_layer : tf.keras.layers.Layer : the keras Layer object 
                        that contains the linear regression coefficients
                        
    """
    #TOFILL
    model = Sequential()
    regression_layer = Dense(1, activation = 'linear', input_dim = input_dim,
                            use_bias = False, kernel_constraint = cstr)
    model.add(regression_layer)
    model.compile(optimizer = "rmsprop", loss = 'mean_squared_error', metrics = ['mse'])
    
    #TOKEEP
    return model, regression_layer

In [ ]:
projected_constraint = ProjectedBoxConstraint(min_value = 0, max_value = 1)

model, regression_layer = linear_regression_model_with_projected_cstrs(A.shape[1], projected_constraint)

model.fit(A,b, epochs=max_epochs, batch_size=batch_size)

In [ ]:
#TOFILL
regression_layer.get_weights()[0].flatten()

In [ ]:
regression_layer.get_weights()[0].sum()

In [ ]:
cvx_cstr_reg_x_opt

As expected the min and max constraints are well satisfied and the coefficient doesn't sum to one. 

Complete the following code to respect the min and max constraints, and rescale the coefficents of the layer so that they sum to one :

In [ ]:
from keras.constraints import Constraint
import keras.backend as K

class ProjectedBoxAndSumConstraint(Constraint):
    def __init__(self, min_value, max_value, sum_value):
        self.min_value = min_value
        self.max_value = max_value
        self.sum_value = sum_value

    def get_config(self):
        return {'min_value': self.min_value,
                'max_value': self.max_value}
    
    #TOFILL
    def __call__(self, w):
        clipped_w = K.clip(w, self.min_value, self.max_value)
        sum_ = tf.reduce_sum(clipped_w)
        return clipped_w / (K.epsilon() + sum_)


In [ ]:
projected_constraint = ProjectedBoxAndSumConstraint(min_value = 0, max_value = 1, sum_value = 1)

model, regression_layer = linear_regression_model_with_projected_cstrs(A.shape[1], projected_constraint)

model.fit(A,b, epochs=max_epochs, batch_size=batch_size)

In [ ]:
#TOFILL
regression_layer.get_weights()[0].flatten()

In [ ]:
regression_layer.get_weights()[0].sum()

In [ ]:
cvx_cstr_reg_x_opt

The training get stuck with a pretty high mean squared error, this can happen as projected gradient algorithms are generally not guaranteed to converge to the optimal solution, even for convex problems, and can have issues with equality constraints as it is the case here. Sometimes penalty functions work better, sometimes projected gradient, both methods are useful to know about.

Again when a problem is convex as here, prefer a convex solver. If you are not sure about the convexity of your problem, try with cvxpy first, if it is not convex it will tell you, and then turn to other methods.

## [Optional] Encoding constraints in the network architecture

(Yet) another way to enforce the constraints is to find a transformation of your parameters that naturally enforces your constraints. In our case the softmax function exactly do what we want : it transforms our real parameters into parameters in $[0,1]$ that sum to 1 :

https://en.wikipedia.org/wiki/Softmax_function

We can then directly opimize the following problem :

<div style="font-size: 150%;" align= "center"> 
\begin{eqnarray*}
    \mathrm{minimize} &\;\;& || A \sigma(x) -b ||^2 \\
\end{eqnarray*}
</div>
    
with no constraints ! To do so, create a custom layer in keras that implements such a linear layer with a softmax transformation of the parameters.

In [ ]:
from keras.layers import Layer

class SoftmaxDense(Layer):
    def __init__(self, units=32):
        super(SoftmaxDense, self).__init__()
        self.units = units

    def build(self, input_shape):  # Create the state of the layer (weights)
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_shape[-1], self.units), dtype="float32"),
            trainable=True,
        )

    def call(self, inputs):  # Defines the computation from inputs to outputs
        softmax = tf.nn.softmax(self.w, axis=0)
        return tf.matmul(inputs, softmax)

    def get_weights(self):
        w = self.w
        output_weights = [tf.nn.softmax(w, axis=0)]
        return output_weights

In [ ]:
def linear_regression_model_with_softmax_as_cstr(input_dim):
    """
    Input : 
    input_dim : int : Number of features of the linear regression
    
    Output : 
    model : tf.keras.Model : the keras Model object
    regression_layer : tf.keras.layers.Layer : the keras Layer object 
                        that contains the linear regression coefficients
                        
    """
    #TOFILL

    model = Sequential()
    regression_layer = SoftmaxDense(1)
    model.add(regression_layer)
    model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error', metrics = ['mse'])
    
    return model, regression_layer

In [ ]:
max_epochs = 5000
batch_size = 10

model, regression_layer = linear_regression_model_with_softmax_as_cstr(A.shape[1])
model.fit(A,b, epochs=max_epochs, batch_size=batch_size)

In [ ]:
from scipy.special import softmax
softmax(model.get_weights()[0])

In [ ]:
#Second solution with the embedding hack
'''
from keras import Model
from keras import backend as K


def linear_regression_model_with_softmax_as_cstr(input_dim):
    """
    Input : 
    input_dim : int : Number of features of the linear regression
    
    Output : 
    model : tf.keras.Model : the keras Model object
    regression_layer : tf.keras.layers.Layer : the keras Layer object 
                        that contains the linear regression coefficients
                        
    """
    #TOFILL
    
    #Features
    a = Input(shape=(input_dim,), name = "a_i")
    
    #Parameters, the softmax enforces the constraints
    #Trick to index only the first line of the embedding :
    zero_constant = K.variable([0], name = "zero_constant", dtype = 'int32')
    zero_index = Input(tensor=zero_constant, dtype='int32', name = 'zero_index')
    x = Embedding(1, input_dim, name="x")(zero_index)
    
    #Softmax layer
    softmax_layer = Activation("softmax")(x)
    
    #Dot product
    y_hat = Dot(axes = -1)([a, softmax_layer])

    model = Model(inputs=[a, zero_index], outputs=y_hat)
    model.compile(loss='mse', optimizer='rmsprop', metrics=["mse"])
    
    #TOKEEP
    return model, x
'''

## [Optional] Implement projected gradient descent in pure numpy

To get a better understanding of the projected gradient descent method, reimplement it yourself with numpy arrays only. Have a look at *Hands on Machine Learning ...* Chapter 4 if you forgot about gradient descent.

First you need to write down the gradient w.r.t $x$ of the linear regression $\nabla_x f_0(x)$. Once you have it you can code the gradient update step, and then project $x$ on the constraints set after each update.

In [ ]:
from numpy.random import normal

x = normal(size = mineral_chemical_compositions.shape[1])

gamma = 0.1
nb_epochs = 1000

for e in range(nb_epochs):
    #TOFILL

    # 1) x gradient update on the whole dataset :


    # 2) Projection of x values on the constraints :
